# Clean updated data and build index
Author: Daheng Wang  
Last modified: 2017-05-19

# Road map
1. Clean updated data
2. Select out non-English tweets
3. Select out original tweets of 'retweet tweets'
4. Build necessary indexes

# Steps

## Initialization

In [10]:
import pymongo, codecs, os, json, multiprocessing, logging, datetime, shelve
from pymongo import IndexModel, ASCENDING, DESCENDING
from pprint import pprint

import mongodb # module for setting up connection with (local) MongoDB database
import multiprocessing_workers # module for splitting workloads between processes
import utilities # module for various custom utility functions
from config import *

In [11]:
updated_data = mongodb.initialize(db_name=DB_NAME, collection_name=UPDATED_COL)

MongoDB on localhost:27017/tweets_ek.c2 connected successfully!


## Clean updated data

### Clean tweets with no user field (probabaly due to server error)
_Step 1_ check how many tweets in updated dataset have no 'user' field, i.e. with no authoer information

In [3]:
if 0 == 1:
    no_user_n = updated_data.count(filter={'user': {'$exists': False}})
    print("Tweets with no 'user' field: {}".format(no_user_n))

CPU times: user 80 ms, sys: 20 ms, total: 100 ms
Wall time: 3min 15s
Tweets with no 'user' field: 0


~~_Step 2_ delete all tweets identified in _Step 1_~~

### Clean server side messages mixed in tweets
_Step 1_ check how many server side message are mixed with tweets in updated database  
Example server side message:
```
{'_id': ObjectId('58be5f60122f5048614f8a2b'), 'limit': {'track': 3, 'timestamp_ms': '1486554726779'}}
```
Server side messages have no 'id' or 'id_str' fields of tweets

In [4]:
if 0 == 1:
    %time messages_n = updated_data.count(filter={'id': {'$exists': False}})
    print('Server side messages: {}'.format(messages_n))

CPU times: user 44 ms, sys: 0 ns, total: 44 ms
Wall time: 1min 59s
Server side messages: 0


~~_Step 2_ delete all server side message identified in _Step 1_~~

## Select out non-English tweets

_Step 1_ check how many tweets are in English/non-English in updated dataset  
English tweets have 'lang' field equals 'en'. Non-english tweets have 'lang' field equals null/'und'/other_lang_identifier.  
See https://dev.twitter.com/overview/api/tweets

In [5]:
if 0 == 1:
    total_tweets_n = updated_data.count()
    en_tweets_n = updated_data.count(filter={'lang': {'$eq': 'en'}})
    print('Total tweets: {}'.format(total_tweets_n))
    print('English tweets: {} ({:.2%})'.format(en_tweets_n, en_tweets_n / total_tweets_n))
    non_en_tweets_n = total_tweets_n - en_tweets_n
    print('non-English tweets: {} ({:.2%})'.format(non_en_tweets_n, non_en_tweets_n / total_tweets_n))

CPU times: user 32 ms, sys: 12 ms, total: 44 ms
Wall time: 1min 45s
Total tweets: 5043587
English tweets: 3754627 (0.7444358548786806)
non-English tweets: 1288960 (0.2555641451213194)


_Step 2_ separate English and non-English tweets into two new collections in MongoDB database

In [8]:
"""
This section generates 2 new collections for English/non-English tweets.
Register EN_UPDATED_COL = 'c2_en' and 
    NONEN_UPDATED_COL = 'c2_nonen' in config if first time.
"""
if 0 == 1:
    # select out English tweets into a new collection
    en_match_dict = {'$match': {'lang': {'$eq': 'en'}}}
    en_out_dict = {'$out': EN_UPDATED_COL}
    en_ppl_lst = [en_match_dict, en_out_dict]
    updated_data.aggregate(pipeline=en_ppl_lst)
    
    # select out non-English tweets into a new collection
    nonen_match_dict = {'$match': {'lang': {'$ne': 'en'}}}
    nonen_out_dict = {'$out': NONEN_UPDATED_COL}
    en_ppl_lst = [nonen_match_dict, nonen_out_dict]
    updated_data.aggregate(pipeline=en_ppl_lst)

CPU times: user 156 ms, sys: 52 ms, total: 208 ms
Wall time: 10min 30s


Check new collection for English updated tweets

In [13]:
en_updated_data = mongodb.initialize(db_name=DB_NAME, collection_name=EN_UPDATED_COL)
en_updated_n = en_updated_data.count()
print('{} English tweets in new collection: {}'.format(en_updated_n, EN_UPDATED_COL))

MongoDB on localhost:27017 connected successfully!
3754627 English tweets in new collection: c2_en


Check new collection for non-English updated tweets

In [14]:
nonen_updated_data = mongodb.initialize(db_name=DB_NAME, collection_name=NONEN_UPDATED_COL)
nonen_updated_n = nonen_updated_data.count()
print('{} non-English tweets in new collection: {}'.format(nonen_updated_n, NONEN_UPDATED_COL))

MongoDB on localhost:27017 connected successfully!
1288960 non-English tweets in new collection: c2_nonen


## Extract out original tweets of 'retweet tweets'

_Step 1_ Check how many 'retweet tweets'.

In [ ]:
# check how many tweets are 'retweet tweets'
if 0 == 1:
    updated_col = mongodb.initialize(db_name=DB_NAME, collection_name=UPDATED_COL)

    total = updated_col.count() # total number of tweets in database
    count = updated_col.count(filter={'retweeted_status': {'$exists': True}})
    print('{} ({:.2%}) tweets are "retweet tweets"'.format(count, (count / total)))

Check how many unique original tweets of 'retweet tweets'.

In [ ]:
# check how many unique original tweets of 'retweet tweets'
if 0 == 1:
    updated_col = mongodb.initialize(db_name=DB_NAME, collection_name=UPDATED_COL)

    unique_rt_origin_id_lst = []

    unique_rt_origin_id_lst = updated_col.distinct('retweeted_status.id',
                                                   filter={'retweeted_status': {'$exists': True}})

    print('{} unique "retweet tweets"'.format(len(unique_rt_origin_id_lst)))

Check how many unique authors of original tweets of 'retweet tweets'.

In [ ]:
# check how many unique authors of original tweets of 'retweet tweets'
if 0 == 1:
    updated_col = mongodb.initialize(db_name=DB_NAME, collection_name=UPDATED_COL)

    rt_origin_unique_user_id_lst = []

    rt_origin_unique_user_id_lst = updated_col.distinct('retweeted_status.user.id',
                                                        filter={'retweeted_status': {'$exists': True}})

    print('{} unique "retweet tweets"'.format(len(rt_origin_unique_user_id_lst)))

_Step 2_ extract out original tweets of 'retweet tweets' into a new collection

In [3]:
"""
This section generates 1 new collections for original tweets of 'retweet tweets'
Register RT_ORIGIN_COL = 'c2_rt_origin' if first time.
"""
if 0 == 1:
    # extract out original tweets of 'retweet tweets' and write to intermediate file
#     match_dict = {'$match': {'retweeted_status': {'$exists': True}}}
    
#     project_dict = {'$project': {'_id': 0, 'retweeted_status': 1}}
    
#     out_dict = {'$out': RT_ORIGIN_COL}
    
#     ppl_lst = [match_dict, project_dict, out_dict]
    
#     updated_data.aggregate(pipeline=ppl_lst)
    
    updated_col = mongodb.initialize(db_name=DB_NAME, collection_name=UPDATED_COL)
    
    cursor = updated_col.find(filter={'retweeted_status': {'$exists': True}},
                              projection={'_id': 0, 'retweeted_status': 1})
    
    with open(os.path.join(TMP_DIR, 'rt_origin.json'), 'w') as f:
        for doc in cursor:
            retweeted_status_dict = doc['retweeted_status']
            f.write(json.dumps(retweeted_status_dict) + '\n')
    print('Done')

MongoDB on localhost:27017/tweets_ek.c2 connected successfully!
Done


In [5]:
if 0 == 1:
    # insert tmp file into new collection
    ! mongoimport --db 'tweets_ek' --collection 'c2_rt_origin' --drop --file ~/Documents/workspace/ibm_tweets-analysis/develop/tmp/rt_origin.json

2017-05-19T22:05:09.539-0400	connected to: localhost
2017-05-19T22:05:09.539-0400	dropping: tweets_ek.c2_rt_origin
2017-05-19T22:05:12.494-0400	[........................] tweets_ek.c2_rt_origin	24.4MB/8.13GB (0.3%)
2017-05-19T22:05:15.494-0400	[........................] tweets_ek.c2_rt_origin	54.3MB/8.13GB (0.7%)
2017-05-19T22:05:18.493-0400	[........................] tweets_ek.c2_rt_origin	79.2MB/8.13GB (1.0%)
2017-05-19T22:05:21.493-0400	[........................] tweets_ek.c2_rt_origin	103MB/8.13GB (1.2%)
2017-05-19T22:05:24.494-0400	[........................] tweets_ek.c2_rt_origin	133MB/8.13GB (1.6%)
2017-05-19T22:05:27.494-0400	[........................] tweets_ek.c2_rt_origin	162MB/8.13GB (1.9%)
2017-05-19T22:05:30.494-0400	[........................] tweets_ek.c2_rt_origin	193MB/8.13GB (2.3%)
2017-05-19T22:05:33.493-0400	[........................] tweets_ek.c2_rt_origin	225MB/8.13GB (2.7%)
2017-05-19T22:05:36.493-0400	[........................] tweets_ek.c2_rt_origin	255MB/8.13G

2017-05-19T22:09:15.493-0400	[######..................] tweets_ek.c2_rt_origin	2.24GB/8.13GB (27.5%)
2017-05-19T22:09:18.494-0400	[######..................] tweets_ek.c2_rt_origin	2.27GB/8.13GB (27.9%)
2017-05-19T22:09:21.493-0400	[######..................] tweets_ek.c2_rt_origin	2.29GB/8.13GB (28.2%)
2017-05-19T22:09:24.494-0400	[######..................] tweets_ek.c2_rt_origin	2.32GB/8.13GB (28.6%)
2017-05-19T22:09:27.494-0400	[######..................] tweets_ek.c2_rt_origin	2.35GB/8.13GB (28.9%)
2017-05-19T22:09:30.494-0400	[######..................] tweets_ek.c2_rt_origin	2.36GB/8.13GB (29.1%)
2017-05-19T22:09:33.493-0400	[#######.................] tweets_ek.c2_rt_origin	2.39GB/8.13GB (29.4%)
2017-05-19T22:09:36.494-0400	[#######.................] tweets_ek.c2_rt_origin	2.42GB/8.13GB (29.8%)
2017-05-19T22:09:39.494-0400	[#######.................] tweets_ek.c2_rt_origin	2.45GB/8.13GB (30.2%)
2017-05-19T22:09:42.493-0400	[#######.................] tweets_ek.c2_rt_origin	2.48GB/8.13G

2017-05-19T22:13:21.493-0400	[#############...........] tweets_ek.c2_rt_origin	4.54GB/8.13GB (55.8%)
2017-05-19T22:13:24.494-0400	[#############...........] tweets_ek.c2_rt_origin	4.57GB/8.13GB (56.1%)
2017-05-19T22:13:27.494-0400	[#############...........] tweets_ek.c2_rt_origin	4.59GB/8.13GB (56.5%)
2017-05-19T22:13:30.494-0400	[#############...........] tweets_ek.c2_rt_origin	4.62GB/8.13GB (56.8%)
2017-05-19T22:13:33.494-0400	[#############...........] tweets_ek.c2_rt_origin	4.65GB/8.13GB (57.2%)
2017-05-19T22:13:36.493-0400	[#############...........] tweets_ek.c2_rt_origin	4.68GB/8.13GB (57.5%)
2017-05-19T22:13:39.493-0400	[#############...........] tweets_ek.c2_rt_origin	4.71GB/8.13GB (57.9%)
2017-05-19T22:13:42.498-0400	[#############...........] tweets_ek.c2_rt_origin	4.74GB/8.13GB (58.2%)
2017-05-19T22:13:45.494-0400	[##############..........] tweets_ek.c2_rt_origin	4.75GB/8.13GB (58.4%)
2017-05-19T22:13:48.494-0400	[##############..........] tweets_ek.c2_rt_origin	4.77GB/8.13G

2017-05-19T22:17:27.494-0400	[####################....] tweets_ek.c2_rt_origin	6.78GB/8.13GB (83.3%)
2017-05-19T22:17:30.494-0400	[####################....] tweets_ek.c2_rt_origin	6.81GB/8.13GB (83.7%)
2017-05-19T22:17:33.494-0400	[####################....] tweets_ek.c2_rt_origin	6.84GB/8.13GB (84.0%)
2017-05-19T22:17:36.493-0400	[####################....] tweets_ek.c2_rt_origin	6.86GB/8.13GB (84.4%)
2017-05-19T22:17:39.493-0400	[####################....] tweets_ek.c2_rt_origin	6.89GB/8.13GB (84.7%)
2017-05-19T22:17:42.493-0400	[####################....] tweets_ek.c2_rt_origin	6.92GB/8.13GB (85.1%)
2017-05-19T22:17:45.493-0400	[####################....] tweets_ek.c2_rt_origin	6.95GB/8.13GB (85.4%)
2017-05-19T22:17:48.494-0400	[####################....] tweets_ek.c2_rt_origin	6.98GB/8.13GB (85.8%)
2017-05-19T22:17:51.494-0400	[####################....] tweets_ek.c2_rt_origin	7.00GB/8.13GB (86.1%)
2017-05-19T22:17:54.494-0400	[####################....] tweets_ek.c2_rt_origin	7.03GB/8.13G

Check how many documents in new collection

In [6]:
if 0 == 1:
    rt_origin_col = mongodb.initialize(db_name=DB_NAME, collection_name=RT_ORIGIN_COL)
    count = rt_origin_col.count()
    print('{} docs in collectionv "{}"'.format(count, RT_ORIGIN_COL))

MongoDB on localhost:27017/tweets_ek.c2_rt_origin connected successfully!
2465388 docs in collectionv "c2_rt_origin"


Check how many unique docs in new collection

In [8]:
if 0 == 1:
    rt_origin_col = mongodb.initialize(db_name=DB_NAME, collection_name=RT_ORIGIN_COL)
    unique_lst = rt_origin_col.distinct('id')
    print('{} unique docs in collectionv "{}"'.format(len(unique_lst), RT_ORIGIN_COL))

MongoDB on localhost:27017/tweets_ek.c2_rt_origin connected successfully!
555635 unique docs in collectionv "c2_rt_origin"


Check how many unique authors of original tweets of "retweet tweets"

In [9]:
if 0 == 1:
    rt_origin_col = mongodb.initialize(db_name=DB_NAME, collection_name=RT_ORIGIN_COL)
    unique_lst = rt_origin_col.distinct('user.id')
    print('{} unique users in collectionv "{}"'.format(len(unique_lst), RT_ORIGIN_COL))

MongoDB on localhost:27017/tweets_ek.c2_rt_origin connected successfully!
143949 unique users in collectionv "c2_rt_origin"


## Build necesary indexes
Build necessy indexes on updated collections to speedup queries.  
This takes quite long time, make sure to set ```background=True``` to run in background.

In [13]:
# IndexModel instances for tweets
id_index = IndexModel([('id', ASCENDING)], background=True)
id_str_index = IndexModel([('id_str', ASCENDING)], background=True)

# IndexModel instances for users
user_id_index = IndexModel([('user.id', ASCENDING)], background=True)
user_id_str_index = IndexModel([('user.id_str', ASCENDING)], background=True)
user_screen_name_index = IndexModel([('user.screen_name', ASCENDING)], background=True)

indexes_list = [id_index, id_str_index, user_id_index, user_id_str_index, user_screen_name_index]

### Build indexes on cleaned updated data collection

In [ ]:
if 0 == 1:
    updated_data.create_indexes(indexes=indexes_list)

### Build indexes on English cleaned updated data collection

In [ ]:
if 0 == 1:
    en_updated_data.create_indexes(indexes=indexes_list)

### Build indexes on non-English cleaned updated data collection

In [ ]:
if 0 == 1:
    nonen_updated_data.create_indexes(indexes=indexes_list)

### Build indexes on original tweets of 'retweet tweets' collection

In [14]:
if 0 == 1:
    rt_origin_col = mongodb.initialize(db_name=DB_NAME, collection_name=RT_ORIGIN_COL)
    rt_origin_col.create_indexes(indexes=indexes_list)

MongoDB on localhost:27017/tweets_ek.c2_rt_origin connected successfully!


### Make sure indexes are created successfully
Check current MongoDB operations on database level

In [ ]:
if 0 == 1:
#     m = importlib.import_module('mongodb')
#     importlib.reload(m)
    
    db = mongodb.initialize_db(db_name=DB_NAME)
    current_ops = db.current_op()
    pprint(current_ops)

List indexes for each collection

In [ ]:
if 0 == 1:
#     pprint(updated_data.index_information())
#     pprint(en_updated_data.index_information())
#     pprint(nonen_updated_data.index_information())
    pprint(rt_origin_col.index_information())